In [60]:
import os
import sys

spark_path = "C:\\Users\\THeo\\Documents\\Kuliah\\Bigdata\\spark-2.1.0-bin-hadoop2.7"
os.environ['SPARK_HOME'] = spark_path
os.environ['HADOOP_HOME'] = spark_path
os.environ['PYSPARK_PYTHON'] = sys.executable

sys.path.append(spark_path + "/bin")
sys.path.append(spark_path + "/python")
sys.path.append(spark_path + "/python/pyspark/")
sys.path.append(spark_path + "/python/lib")
sys.path.append(spark_path + "/python/lib/pyspark.zip")
sys.path.append(spark_path + "/python/lib/py4j-0.10.4-src.zip")

from pyspark import SparkContext
from pyspark import SparkConf

conf = SparkConf()
conf.set("spark.executor.memory", "4g")
conf.set("spark.cores.max", "4")

sc = SparkContext("local", conf=conf)
print (sc)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local) created by __init__ at <ipython-input-2-d8d32de5396b>:23 

In [34]:
airlines=sc.textFile('C:\\Users\\THeo\\Documents\\Kuliah\\Bigdata\\dataset\\airlines.csv')
flights=sc.textFile('C:\\Users\\THeo\\Documents\\Kuliah\\Bigdata\\dataset\\flights.csv')
airports=sc.textFile('C:\\Users\\THeo\\Documents\\Kuliah\\Bigdata\\dataset\\airports.csv')

flights.take(10)

['2014-04-01,19805,1,JFK,LAX,0854,-6.00,1217,2.00,355.00,2475.00',
 '2014-04-01,19805,2,LAX,JFK,0944,14.00,1736,-29.00,269.00,2475.00',
 '2014-04-01,19805,3,JFK,LAX,1224,-6.00,1614,39.00,371.00,2475.00',
 '2014-04-01,19805,4,LAX,JFK,1240,25.00,2028,-27.00,264.00,2475.00',
 '2014-04-01,19805,5,DFW,HNL,1300,-5.00,1650,15.00,510.00,3784.00',
 '2014-04-01,19805,6,OGG,DFW,1901,126.00,0640,95.00,385.00,3711.00',
 '2014-04-01,19805,7,DFW,OGG,1410,125.00,1743,138.00,497.00,3711.00',
 '2014-04-01,19805,8,HNL,DFW,1659,4.00,0458,-22.00,398.00,3784.00',
 '2014-04-01,19805,9,JFK,LAX,0648,-7.00,1029,19.00,365.00,2475.00',
 '2014-04-01,19805,10,LAX,JFK,2156,21.00,0556,1.00,265.00,2475.00']

In [35]:
from datetime import datetime
from collections import namedtuple

In [36]:
fields = ('date','airline','flightnum','origin','dest','dep','dep_delay','arv','arv_delay','airtime','distance')

Flight = namedtuple('Flight',fields,verbose=True)

DATE_FMT = "%Y-%m-%d"
TIME_FMT = "%H%M"

from builtins import property as _property, tuple as _tuple
from operator import itemgetter as _itemgetter
from collections import OrderedDict

class Flight(tuple):
    'Flight(date, airline, flightnum, origin, dest, dep, dep_delay, arv, arv_delay, airtime, distance)'

    __slots__ = ()

    _fields = ('date', 'airline', 'flightnum', 'origin', 'dest', 'dep', 'dep_delay', 'arv', 'arv_delay', 'airtime', 'distance')

    def __new__(_cls, date, airline, flightnum, origin, dest, dep, dep_delay, arv, arv_delay, airtime, distance):
        'Create new instance of Flight(date, airline, flightnum, origin, dest, dep, dep_delay, arv, arv_delay, airtime, distance)'
        return _tuple.__new__(_cls, (date, airline, flightnum, origin, dest, dep, dep_delay, arv, arv_delay, airtime, distance))

    @classmethod
    def _make(cls, iterable, new=tuple.__new__, len=len):
        'Make a new Flight object from a sequence or iterable'
        result = new(cls, iterable)
        if len(result) != 11:
  

In [40]:
def parse(row):
    row[0] = datetime.strptime(row[0], DATE_FMT).date()
    row[5] = datetime.strptime(row[5], TIME_FMT).time()
    row[6] = float(row[6])
    row[7] = datetime.strptime(row[7], TIME_FMT).time()
    row[8] = float(row[8])
    row[9] = float(row[9])
    row[10]= float(row[10])
    return Flight(*row[:11])



In [45]:
flightsParsed = flights.map(lambda x: x.split(',')).map(parse)

In [46]:
flightsParsed.first()

Flight(date=datetime.date(2014, 4, 1), airline='19805', flightnum='1', origin='JFK', dest='LAX', dep=datetime.time(8, 54), dep_delay=-6.0, arv=datetime.time(12, 17), arv_delay=2.0, airtime=355.0, distance=2475.0)

In [48]:
flightsParsed.map(lambda x:x.distance)

PythonRDD[41] at RDD at PythonRDD.scala:48

In [50]:
totalDistance=flightsParsed.map(lambda x:x.distance).reduce(lambda x,y:x+y)
avgDistance=totalDistance/flightsParsed.count()
print (avgDistance)

794.8585013871385


In [53]:
flightsParsed.filter(lambda x:x.dep_delay>0).count()/float(flightsParsed.count())

0.3753871510922012

In [54]:
flightsParsed.persist()

PythonRDD[47] at RDD at PythonRDD.scala:48

In [57]:
sumCount=flightsParsed.map(lambda x:x.dep_delay).aggregate((0,0),
                                                           (lambda some, value: (some[0] + value, some[1]+1)),
                                                           (lambda some1,some2: (some1[0]+some2[0],some1[1]+some2[1])))


print (sumCount)

(3964730.0, 476881)


In [58]:
sumCount[0]/float(sumCount[1])

8.313877046894298

In [59]:
flightsParsed.map(lambda x: int(x.dep_delay/60)).countByValue()

defaultdict(int,
            {0: 452963,
             1: 16016,
             2: 4893,
             3: 1729,
             4: 701,
             5: 249,
             6: 113,
             7: 66,
             8: 43,
             9: 26,
             10: 15,
             11: 12,
             12: 9,
             13: 15,
             14: 13,
             15: 4,
             17: 2,
             20: 4,
             21: 3,
             24: 3,
             25: 1,
             28: 1})